# Overview

This notebook imports raw ws3 input data, reformats and monkey-patches the data, and exports Woodstock formatted input data files (which we will use in other DSS notebooks for this case as the input data files). 

# Set up environment

In [1]:
%load_ext autoreload
%autoreload 2

In [81]:
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import ws3.forest, ws3.core
import csv
import numpy as np
import time
from functools import partial, wraps

Define some key model parameters (will get used but defined here up top for convenience).

In [3]:
period_length = 10
max_age =  1000

In [4]:
# Input paths
vdyp_curves_smooth_tsa04_path = './data/vdyp_curves_smooth-tsa04.feather'

# Output paths
stands_csv_path = './data/stands.csv'

stands_mdf_csv_path = './data/stands_mdf.csv'

yld_vdyp_csv_path = './data/yld_vdyp.csv'

yldmerged_csv_path = './data/yldmerged.csv'

woodstock_model_files_lan_path = './data/woodstock_model_files/tsa04.lan'
woodstock_model_files_are_path = './data/woodstock_model_files/tsa04.are'
woodstock_model_files_yld_path = './data/woodstock_model_files/_tsa04.yld'
woodstock_model_files_act_path = './data/woodstock_model_files/tsa04.act'
woodstock_model_files_trn_path = './data/woodstock_model_files/tsa04.trn'

# Import and reformat inventory and yield input data

Read forest inventory data into memory (vector polygon GIS data layer with attribute table, in ESRI Shapefile format). This dataset represents a small subset of timber supply area (TSA) 17 in British Columbia. We monkey-patch the inventory data here to make it line up nicely with what we need downstream as input for the ws3 model (i.e., changes we make here to the in-memory dataset are not saved to the original dataset on disk). Most of what we are doing here is setting up the _theme_ columns in the attribute table, which should help newer ws3 users make the connection between input data and the landscape themes in ws3 model further down.

In [5]:
Start = time.time()
stands = gpd.read_file('data/tsa04contclass/tsa04.shp')
print('It took', round((time.time() - Start) / 60, 1), "minutes to run this script.")

It took 11.9 minutes to run this script.


Import CANFI tree species lookup table (associates tree species names with integer numerical values, which we use as theme data values in the ws3 model), and insert species code values into the yield curve dataframe.

In [6]:
stands

,FID_Golden,FID_Gold_1,ZONE,SUBZONE,VARIANT,PHASE,NTRLDSTRBN,MAP_LABEL,FID_Gold_2,HERD_STATU,...,ORIG_FID,Shape_Leng,Shape_Area,contclass,rollup,netdown,THLB_Area,Age_2023,AU,geometry
0,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,892.970730,16474.221678,C,THLB,THLB,1.647422,46,6,"POLYGON ((651330.000 1473337.500, 651330.000 1..."
1,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,847.015427,25214.104031,C,THLB,THLB,2.521410,46,6,"POLYGON ((651251.113 1473060.000, 651250.257 1..."
2,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,2146.240646,87035.420516,C,THLB,THLB,8.703542,46,6,"POLYGON ((651090.000 1473000.000, 651090.000 1..."
3,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,468,255.813594,2305.734220,C,THLB,THLB,0.230573,46,6,"POLYGON ((650746.974 1472340.000, 650640.000 1..."
4,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,469,859.337139,15510.057930,C,THLB,THLB,1.551006,46,6,"POLYGON ((651390.000 1473330.000, 651390.000 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63203,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,71281,326.907495,1502.856584,N,6_Riparian,6_01_Stream_Buffer,0.000000,83,33,"POLYGON ((649003.440 1506715.961, 649006.877 1..."
63204,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,71281,476.065471,1950.527358,N,6_Riparian,6_01_Stream_Buffer,0.000000,83,33,"POLYGON ((648270.000 1506662.850, 648270.000 1..."
63205,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,71281,415.325791,1606.933608,N,6_Riparian,6_01_Stream_Buffer,0.000000,83,33,"POLYGON ((649050.000 1506509.390, 649050.000 1..."
63206,1,11,MH,mm,2,None,NDT1,MHmm2,-1,None,...,71292,287.561926,1279.447945,N,6_Riparian,6_01_Stream_Buffer,0.000000,101,33,"POLYGON ((622490.804 1495979.904, 622468.035 1..."


In [102]:
def stratify_stand(r, lexmatch=False, lexmatch_fieldname_suffix='_lexmatch'):
    result = ''
    if lexmatch:
        result += 3 * r['BEC_ZONE_C%s' % lexmatch_fieldname_suffix]
        result += '_'
        #result += r.BCLCS_LEVEL_5
        #result += '_'
        result += 2 * r['SPECIES_CD%s' % lexmatch_fieldname_suffix]
        if r.BCLCS_LEVE == 'TM' and r.SPECIES_CD_2 != None:
            result += '+' + r['SPECIES__1%s' % lexmatch_fieldname_suffix]
    else:
        result += r.BEC_ZONE_C
        result += '_'
        #result += r.BCLCS_LEVEL_5
        #result += '_'
        result += r.SPECIES_CD
        # if r.BCLCS_LEVEL_4 == 'TM' and r.SPECIES_CD_2 != None:
        if r.BCLCS_LEVE == 'TM' and r.SPECIES_CD_2 != None:
            result += '+' + r.SPECIES_CD_2
    return result

In [104]:
stands['BEC_ZONE_C_lexmatch'] = stands.BEC_ZONE_C.str.ljust(4, fillchar='x')
stands['SPECIES_CD_lexmatch'] = stands['SPECIES_CD'].str.ljust(4, 'x')
stands['SPECIES_CD_lexmatch'] = stands['SPECIES_CD'].str[:1] + stands['SPECIES_CD']

for i in range(1, 2):
    stands['SPECIES_CD_%i_lexmatch' % i] = stands['SPECIES__%i' % i].str.ljust(4, 'x')
    stands['SPECIES_CD_%i_lexmatch' % i] = stands['SPECIES__%i' % i].str[:1] + stands['SPECIES__%i' % i]

stratify_stand = stratify_stand
stratify_stand_lexmatch = partial(stratify_stand, lexmatch=True)

stands['stratum'] = stands.apply(stratify_stand, axis=1)
stands['stratum_lexmatch'] = stands.apply(stratify_stand_lexmatch, axis=1)
stands

,FID_Golden,FID_Gold_1,ZONE,SUBZONE,VARIANT,PHASE,NTRLDSTRBN,MAP_LABEL,FID_Gold_2,HERD_STATU,...,THLB_Area,Age_2023,AU,geometry,stratum,BEC_ZONE_CODE_lexmatch,SPECIES_CD_lexmatch,SPECIES_CD_1_lexmatch,BEC_ZONE_C_lexmatch,stratum_lexmatch
0,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,1.647422,46,6,"POLYGON ((651330.000 1473337.500, 651330.000 1...",SWB_SX,SWBx,SSX,PPL,SWBx,SWBxSWBxSWBx_SSXSSX
1,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,2.521410,46,6,"POLYGON ((651251.113 1473060.000, 651250.257 1...",SWB_SX,SWBx,SSX,PPL,SWBx,SWBxSWBxSWBx_SSXSSX
2,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,8.703542,46,6,"POLYGON ((651090.000 1473000.000, 651090.000 1...",SWB_SX,SWBx,SSX,PPL,SWBx,SWBxSWBxSWBx_SSXSSX
3,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,0.230573,46,6,"POLYGON ((650746.974 1472340.000, 650640.000 1...",SWB_SX,SWBx,SSX,PPL,SWBx,SWBxSWBxSWBx_SSXSSX
4,1,1,SWB,uns,None,None,NDT2,SWBuns,-1,None,...,1.551006,46,6,"POLYGON ((651390.000 1473330.000, 651390.000 1...",SWB_SX,SWBx,SSX,PPL,SWBx,SWBxSWBxSWBx_SSXSSX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63203,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,0.000000,83,33,"POLYGON ((649003.440 1506715.961, 649006.877 1...",BWBS_AT,BWBS,AAT,SSX,BWBS,BWBSBWBSBWBS_AATAAT
63204,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,0.000000,83,33,"POLYGON ((648270.000 1506662.850, 648270.000 1...",BWBS_AT,BWBS,AAT,SSX,BWBS,BWBSBWBSBWBS_AATAAT
63205,1,7,BWBS,dk,None,None,NDT3,BWBSdk,-1,None,...,0.000000,83,33,"POLYGON ((649050.000 1506509.390, 649050.000 1...",BWBS_AT,BWBS,AAT,SSX,BWBS,BWBSBWBSBWBS_AATAAT
63206,1,11,MH,mm,2,None,NDT1,MHmm2,-1,None,...,0.000000,101,33,"POLYGON ((622490.804 1495979.904, 622468.035 1...",MH_AT,MHxx,AAT,HHM,MHxx,MHxxMHxxMHxx_AATAAT


In [7]:
canfi_map = {'AC':1211, 
             'AT':1201, 
             'BL':304, 
             'EP':1303, 
             'FDI':500, 
             'HW':402,
             'HM':403,
             'PL':204, 
             'PLI':204, 
             'SB':101, 
             'SE':104, 
             'SW':105, 
             'SX':100,
             'S':100,
            'AT+SX':1201,
            'SX+AT':100}

In [8]:
Aspen = ['AC', 'ACT', 'AT', 'EP', 'VB', 'MB', 'AT+SX']
Bal = ['B', 'BA', 'BG', 'BL']
Cedar = ['CW', 'YC']
Alder = ['D', 'DR']
DougFir = ['F', 'FD', 'FDC', 'FDI']
Hem = ['H', 'HM', 'HW']
Pine = ['PA', 'PL', 'PLC', 'PW', 'PLI', 'PY']
Spruce = ['S', 'SS', 'SW', 'SX', 'SE', 'SXW', 'SB', 'SX+AT']

In [10]:
columns_to_keep = ['TSA_NUMBER', 'contclass', 'Age_2023', 'AU', 'geometry', 'SPECIES_CD']
stands_mdf = stands[columns_to_keep].copy()
stands_mdf.loc[:,'area'] = stands_mdf.geometry.area * 0.0001 # monkey-patch broken area attribute
stands_mdf =  stands_mdf.rename(columns={'TSA_NUMBER': 'theme0', 'contclass':'theme1', 'AU':'theme2', 'Age_2023':'age', 'SPECIES_CD':'species'})
stands_mdf['theme0'] = stands_mdf['theme0'].replace({'04': 'tsa04'})
stands_mdf['theme1'] = stands_mdf['theme1'].replace({'C': 1, 'N': 0})
stands_mdf = stands_mdf.drop(columns='geometry')
stands_mdf.insert(4, 'theme3',  stands_mdf['species'].map(canfi_map)) #Burn CANFI species codes into stand data
stands_mdf['theme3'] ggg= stands_mdf['theme3'].astype(int)
stands_mdf.drop(columns=['species'], inplace=True)
stands_mdf.insert(5, 'theme4', stands_mdf['theme2']) # to be filled out with the scpecies code
stands_mdf.insert(5, 'age', stands_mdf.pop('age'))
stands_mdf

,theme0,theme1,theme2,theme3,theme4,age,area
0,tsa04,1,6,100,6,46,1.647422
1,tsa04,1,6,100,6,46,2.521410
2,tsa04,1,6,100,6,46,8.703542
3,tsa04,1,6,100,6,46,0.230573
4,tsa04,1,6,100,6,46,1.551006
...,...,...,...,...,...,...,...
63203,tsa04,0,33,1201,33,83,0.150252
63204,tsa04,0,33,1201,33,83,0.195038
63205,tsa04,0,33,1201,33,83,0.160693
63206,tsa04,0,33,1201,33,101,0.127911


Read yield data from a CSV file and recast AU column data type to integer.

In [61]:
print('whole area (ha) is:', stands_mdf['area'].sum())
print('whole contributing area (ha) is:', stands_mdf[stands_mdf['theme1'] == 1]['area'].sum())
print('whole non-contributing area (ha) is:', stands_mdf[stands_mdf['theme1'] == 0]['area'].sum())

whole area (ha) is: 191273.58586001678
whole contributing area (ha) is: 134732.25775766742
whole non-contributing area (ha) is: 56541.328102349355


Burn CANFI species codes into yield data tables.

In [62]:
def canfi_species(stratum_code):
    s = stratum_code.split('_')[-1].split('+')[0]
    result = canfi_map[s]
    return result

In [67]:
yld_vdyp = pd.read_feather(vdyp_curves_smooth_tsa04_path)
yld_vdyp['canfi_species'] = yld_vdyp.apply(lambda row: canfi_species(row['stratum_code']), axis=1).astype(int)
yld_vdyp['au_vdyp'] = pd.factorize(yld_vdyp['stratum_code'] + '_' + yld_vdyp['si_level'])[0] + 1
yld_vdyp

,index,age,volume,stratum_code,si_level,canfi_species,au_vdyp
0,17,18,6.282000e-11,ESSF_BL,L,304,1
1,18,19,3.625496e-08,ESSF_BL,L,304,1
2,19,20,7.916861e-07,ESSF_BL,L,304,1
3,20,21,6.105747e-06,ESSF_BL,L,304,1
4,21,22,2.801684e-05,ESSF_BL,L,304,1
...,...,...,...,...,...,...,...
12296,294,295,9.945361e+01,BWBS_AT,H,1201,42
12297,295,296,9.878314e+01,BWBS_AT,H,1201,42
12298,296,297,9.811513e+01,BWBS_AT,H,1201,42
12299,297,298,9.744960e+01,BWBS_AT,H,1201,42


In [68]:
# This is for VDYP yield curve

# Create a new column 'id' with numeric values representing unique combinations of 'stratum_code' and 'si_level'
# yld_vdyp['au_vdyp'] = pd.factorize(yld_vdyp['stratum_code'] + '_' + yld_vdyp['si_level'])[0] + 1

# Initialize the 'au_id_vdyp' column in yld_vdyp DataFrame
# yld_vdyp['au_id_vdyp'] = 0

# scsi_au = {'04': {}}
# ria_tsas = ['04']
# for tsa in ria_tsas:
#     vdyp_curves_ = yld_vdyp.set_index(['stratum_code', 'si_level'])
#     for stratum_code, si_level in list(vdyp_curves_.index.unique()):
#         # Access a value from the dictionary 'scsi_au' using 'tsa' as the key 
#         # and '(stratum_code, si_level)' as the second key
#         au_id_ = scsi_au['04'][(stratum_code, si_level)]
        
#         # Calculate 'au_id' by multiplying 100000 with the integer value of 'tsa' and then adding 'au_id_'
#         au_id = 100000 * int(tsa) + au_id_
        
#         # Assign the calculated 'au_id' directly into the 'au_id_vdyp' column based on the condition
#         yld_vdyp.loc[(yld_vdyp['stratum_code'] == stratum_code) & 
#                      (yld_vdyp['si_level'] == si_level), 'au_id_vdyp'] = au_id

# yld_vdyp

In [ ]:
# yld_vdyp['AU'] = yld_vdyp.apply(calculate_au, axis=1)
# yld_vdyp['AU'] = yld_vdyp['AU'].astype(int)
# yld_vdyp

In [70]:
bec_zone_stands = stands['ZONE'].drop_duplicates()
bec_zone_stands

0        SWB
28      BWBS
388       MH
1108     SBS
1474    ESSF
2572     CWH
2812    BAFA
2916     CMA
Name: ZONE, dtype: object

In [72]:
# bec_zone_vdyp = yld_vdyp['zone'].drop_duplicates()
# bec_zone_vdyp

In [73]:
au_stands = stands['AU'].drop_duplicates()
len(au_stands), au_stands

(28,
 0         6
 10       22
 12       13
 30       29
 56       33
 96       19
 146      10
 388      26
 412       3
 498      25
 802       7
 922      31
 1154     18
 1700     24
 1912     23
 2268      1
 2890     30
 3828     12
 4244      5
 4504     16
 5026     21
 7412      4
 12893     2
 14077    17
 27490    28
 28596    14
 28928    15
 34010    32
 Name: AU, dtype: int64)

In [75]:
# au_vdyp = yld_vdyp['AU'].drop_duplicates()
# len(au_vdyp), au_vdyp

Create analysis unit (AU) dataframe from stands dataframe data.

In [ ]:
AU = pd.DataFrame(stands_mdf['theme2']).drop_duplicates()
AU.rename(columns={'theme2':'AU'}, inplace=True)

Join `AU` and `yld_vdyp` dataframes.

In [ ]:
yldmerged = pd.merge(AU, yld_vdyp, on=['AU'], how='inner')
yldmerged

In [ ]:
au_yldmerged = yldmerged['AU'].drop_duplicates()
len(au_yldmerged), au_yldmerged

Add a new `curve_id` colume that has same data values as `AU` column.

In [ ]:
yldmerged['curve_id'] = yldmerged['AU'] 

Save reformatted data to CSV files. 

In [76]:
stands_mdf.to_csv(stands_mdf_csv_path, index=False)
yld_vdyp.to_csv(yld_vdyp_csv_path, index=False)
yldmerged.to_csv(yldmerged_csv_path, header=True, index=False)
stands.to_csv(stands_csv_path, header=True, index=False)

Rename stuff to match variable names we expect further down.

In [ ]:
stands_table = stands_mdf
curve_points_table = yldmerged
# curve_points_table = pd.read_csv("data/yldmerged_mmdf.csv")
curve_points_table.set_index('AU', inplace=True)

# Export Woodstock-formatted input files 

We can use the new ws3 model instance we just built to export ws3 input files in Woodstock file format. We do this for three reasons. 

The first reason is that it will be simpler and more compact in the actual DSS notebook to instantiate the `ForestModel` object from these Woodstock-formatted files (and also this will provide an opportunity to demonstrate the existance and usage of the Woodstock model import functions that are built into ws3). 

The second reason is that the process of exporting data from a live `ws3.forest.ForestModel` instance to Woodstock-formatted input data files provides some insight into the internal structure and workings of ws3 models (which can be a challenging thing to get started with, particularly if you do not have a lot of experience building and running forest estate models). 

The third reason is that Woodstock file format is designed to be "human readable" (sort of... nobody ever said it would be super easy or super fun). Picking through the exported Woodstock-formatted files might help some people better understand the structure and details of the model we have built. If you have no experience reading Woodstock-formatted model input data files, then this is going to be trickier (unless you pause here and go take an introductory Woodstock training course of sort). Many forest professionals already have familiarity with Woodstock software and its special file format (through having been exposed to this at some point in their career). 

Start by creating a new subdirectory to hold the new Woodstock-formatted data files.

In [ ]:
!mkdir data/woodstock_model_files

## LANDSCAPE section

The LANDSCAPE section defines stratification variables (themes) and stratification variable values (basecodes). 

In [ ]:
theme_cols=['theme0', # TSA 
            'theme1', # THLB
            'theme2', # AU
            'theme3', # leading species code
            'theme4'  # yield curve ID
           ]
basecodes = [list(map(lambda x: str(x), stands_table[tc].unique())) for tc in theme_cols]
basecodes[2] = list(set(basecodes[2] + list(stands_table['theme2'].astype(str))))
basecodes[3] = list(set(basecodes[3] + list(stands_table['theme3'].astype(str))))
basecodes[4] = list(set(basecodes[4] + list(stands_table['theme4'].astype(str))))

In [ ]:
with open(woodstock_model_files_lan_path, 'w') as file:
    print('*THEME Timber Supply Area (TSA)', file=file)
    print('tsa04',file=file)
    print('*THEME Timber Harvesting Land Base (THLB)', file=file)
    for basecode in basecodes[1]: print(basecode, file=file)
    print('*THEME Analysis Unit (AU)', file=file)
    for basecode in basecodes[2]: print(basecode, file=file)
    print('*THEME Leading tree species (CANFI species code)', file=file)
    for basecode in basecodes[3]: print(basecode, file=file)
    print('*THEME Yield curve ID', file=file)
    for basecode in basecodes[4]: print(basecode, file=file)

## AREAS section

The AREAS section defines the initial forest inventory, in terms of how many hectares of which age class are present in which development type (where a development type is defined as a unique sequence of landscape theme variable values).

In [ ]:
gstands = stands_table.groupby(theme_cols+['age'])

In [ ]:
with open(woodstock_model_files_are_path, 'w') as file:
    for name, group in gstands:
        dtk, age, area = tuple(map(lambda x: str(x), name[:-1])), int(name[-1]), group['area'].sum()
        print('*A', ' '.join(v for v in dtk), age, area, file=file)

## YIELDS section

The YIELDS section defines yield curves (in this example we only track merchantable log volume, but we can use yield curves to track all sorts of other stuff). 

In [ ]:
with open(woodstock_model_files_yld_path, 'w') as file:
    tot=[]
    swd=[]
    hwd=[]
    unique_au_rows = curve_points_table[~curve_points_table.index.duplicated(keep='first')]    
    for AU, au_row in unique_au_rows.iterrows():
        yname = 's%04d' % int(au_row.canfi_species)    
        curve_id = au_row.curve_id
        mask = ('?', '?', str(AU), '?', str(curve_id))
        points = [(r.age, r.volume) for _, r in curve_points_table.loc[AU].iterrows() if not r.age % period_length and r.age <= max_age]
        c = ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
        print('*Y', ' '.join(v for v in mask), file=file)
        print(yname, '1', ' '.join(str(int(c[x])) for x in range(0, 300, 10)), file=file)
        if yname not in tot:
            tot.append(yname)
        if int(au_row.canfi_species) > 1200:
            if yname not in hwd: hwd.append(yname)
        else:
            if yname not in swd: swd.append(yname)
    print('*YC ? ? ? ? ?', file=file)
    print('totvol _SUM(%s)' % ', '.join(map(str, tot)), file=file)
    print('swdvol _SUM(%s)' % ', '.join(map(str, swd)), file=file)
    print('hwdvol _SUM(%s)' % ', '.join(map(str, hwd)), file=file)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

cvol = c
ccai = c.cai()
cmai = c.mai()
cmaiytp = c.mai().ytp()
x_cmai = cmaiytp.lookup(0) # optimal rotation age (i.e., culmination of MAI curve)
labels = 'total volume', 'MAI (and CAI)', 'MAI YTP'

ax[0].plot(*zip(*c.points()))
ax[0].plot([0, x_cmai], [0., cvol[x_cmai]], linestyle='--', color='green')

ax[1].plot(*zip(*c.mai().points()))
ax[1].plot(*zip(*c.cai().points()), linestyle=':')

ax[2].plot(*zip(*c.mai().ytp().points()))
ax[2].axhline(0, color='black')

for i in range(len(ax)):
    ax[i].set_ylabel(labels[i])
    ax[i].set_ylim(0, None)
    ax[i].axvline(x_cmai, color='red')
plt.xlim(0, 300)

## ACTIONS section

The ACTIONS section defines actions that can be applied in the model (e.g., harvesting, planting, thinning, fertilization, etc). 

In [ ]:
with open(woodstock_model_files_act_path, 'w') as file:
    print('ACTIONS', file=file)
    print('*ACTION harvest Y', file=file)
    print('*OPERABLE harvest', file=file)
    print('? 1 ? ? ? _AGE >= 140 AND _AGE <= 600', file=file)

## TRANSITIONS section

The TRANSITIONS section defines transitions (i.e., transition to a new development type and age class induced by applying a specific action to a specific combination of development type and age class). If there were no transitions in a forest estate model, it would simply be aging (i.e., growing) the forest forward from time step 1 through to time step N.

In [ ]:
with open(woodstock_model_files_trn_path, 'w') as file:
    acode = 'harvest'
    print('*CASE', acode, file=file)
    record_au = set()
    for au_id, au_row in stands_table.iterrows():
        if au_row.theme2 in record_au: continue
        if not au_row.theme1: continue
        target_curve_id = au_row.theme4  
        smask = ' '.join(('?', '?' , str(target_curve_id), '?', '?'))
        tmask = ' '.join(('?', '?' , '?', '?', str(target_curve_id)))
        print('*SOURCE', smask, file=file)
        print('*TARGET', tmask, '100', file=file)
        record_au.add(au_row.theme2)

In [ ]:
stands_table = stands_mdf
curve_points_table = pd.read_csv("data/yldmerged_mmdf.csv")
curve_points_table.set_index('AU', inplace=True)

In [ ]:
with open(woodstock_model_files_yld_path, 'w') as file:
    tot=[]
    swd=[]
    hwd=[]
    unique_au_rows = curve_points_table[~curve_points_table.index.duplicated(keep='first')]    
    for AU, au_row in unique_au_rows.iterrows():
        yname = 's%04d' % int(au_row.canfi_species)    
        curve_id = au_row.curve_id
        mask = ('?', '?', str(AU), '?', str(curve_id))
        points = [(r.age, r.volume) for _, r in curve_points_table.loc[AU].iterrows() if not r.age % period_length and r.age <= max_age]
        c = ws3.core.Curve(yname, points=points, type='a', is_volume=True, xmax=max_age, period_length=period_length)
        print('*Y', ' '.join(v for v in mask), file=file)
        print(yname, '1', ' '.join(str(int(c[x])) for x in range(0, 300, 10)), file=file)
        if yname not in tot:
            tot.append(yname)
        if int(au_row.canfi_species) > 1200:
            if yname not in hwd: hwd.append(yname)
        else:
            if yname not in swd: swd.append(yname)
    print('*YC ? ? ? ? ?', file=file)
    print('totvol _SUM(%s)' % ', '.join(map(str, tot)), file=file)
    print('swdvol _SUM(%s)' % ', '.join(map(str, swd)), file=file)
    print('hwdvol _SUM(%s)' % ', '.join(map(str, hwd)), file=file)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 12), sharex=True)

cvol = c
ccai = c.cai()
cmai = c.mai()
cmaiytp = c.mai().ytp()
x_cmai = cmaiytp.lookup(0) # optimal rotation age (i.e., culmination of MAI curve)
labels = 'total volume', 'MAI (and CAI)', 'MAI YTP'

ax[0].plot(*zip(*c.points()))
ax[0].plot([0, x_cmai], [0., cvol[x_cmai]], linestyle='--', color='green')

ax[1].plot(*zip(*c.mai().points()))
ax[1].plot(*zip(*c.cai().points()), linestyle=':')

ax[2].plot(*zip(*c.mai().ytp().points()))
ax[2].axhline(0, color='black')

for i in range(len(ax)):
    ax[i].set_ylabel(labels[i])
    ax[i].set_ylim(0, None)
    ax[i].axvline(x_cmai, color='red')
plt.xlim(0, 300)